In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
import re 
import nltk 
from nltk import word_tokenize
from nltk.tokenize import WordPunctTokenizer , word_tokenize , sent_tokenize

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier


In [5]:
import joblib

In [6]:
from sklearn import model_selection

In [7]:
from imblearn.pipeline import Pipeline, make_pipeline
from imblearn.over_sampling import SMOTE

In [8]:
os.chdir('D:/Projects/Yaaota Task/scrapy-tutorial-starter/tutorial')

In [9]:
db = sqla.create_engine('sqlite:///ecommerce.db')
data = pd.read_sql('select * from products',db)

In [10]:
data.head()

,URL,TITLE,MAIN_CATEGORY
0,https://2b.com.eg/en/kioxia-transmemory-u202w-...,KIOXIA TransMemory U202W 16GB LU202W016GG4,Computers
1,https://2b.com.eg/en/a-data-uv250-usb-flash-dr...,A Data UV250 USB Flash Drive 32 GB - Silver,Computers
2,https://2b.com.eg/en/adata-c008-capless-slidin...,A Data C008 Capless Sliding USB Flash Drive - ...,Computers
3,https://2b.com.eg/en/hikvision-usb-flash-drive...,Hikvision USB Flash Drive M200 STD - 16GB,Computers
4,https://2b.com.eg/en/tp-link-tl-wa855re-300mbp...,TP-Link Wi-Fi Range Extender 300Mbps (TL-WA855RE),Computers


In [11]:
type(data)

pandas.core.frame.DataFrame

In [12]:
data.shape

(3745, 3)

In [13]:
data.describe()

,URL,TITLE,MAIN_CATEGORY
count,3745,3745,3745
unique,3626,3603,5
top,https://2b.com.eg/en/2b-ecstasy-gaming-mid-tow...,"L'avvento Casual Daypack Backpack, Lightweight...",Accessories
freq,2,16,2274


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3745 entries, 0 to 3744
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   URL            3745 non-null   object
 1   TITLE          3745 non-null   object
 2   MAIN_CATEGORY  3745 non-null   object
dtypes: object(3)
memory usage: 87.9+ KB


In [14]:
data['URL'].value_counts()

https://2b.com.eg/en/2b-ecstasy-gaming-mid-tower-case-ecstasy-gaming-power-supply-650w-80plus-ecstasy-gaming-triple-fan-case.html    2
https://2b.com.eg/en/copper-cable-for-cctv-av-and-power-15m-with-power-and-av-connecors.html                                         2
https://2b.com.eg/en/spirit-of-gamer-airblade-600-blue-17-cooler.html                                                                2
https://2b.com.eg/en/hp-omen-encoder-gaming-keyboard-6yw75aa-brown-switch.html                                                       2
https://2b.com.eg/en/white-shark-gaming-headset-gh-2042-ocelot-black.html                                                            2
                                                                                                                                    ..
https://2b.com.eg/en/honor-magic-watch-2-minos-brown.html                                                                            1
https://2b.com.eg/en/sandisk-cruzer-blade-usb-flash-dri

In [16]:
data['MAIN_CATEGORY'].value_counts()

Accessories        2274
Home Appliances     424
Computers           388
Mobile & Tablet     355
Gaming              304
Name: MAIN_CATEGORY, dtype: int64

#### Here we have some observations:
<ol>
    <li>No Missing data</li>
    <li>Data is imbalanced for Accessories category</li>
    <li>There are some tiny duplicated values in url or title we will try to check if we can drop the record with MAIN_CATEGORY = Accessories</li>
</ol>

In [17]:
data.query('URL == "https://2b.com.eg/en/2b-ecstasy-gaming-mid-tower-case-ecstasy-gaming-power-supply-650w-80plus-ecstasy-gaming-triple-fan-case.html" and MAIN_CATEGORY=="Accessories"')

,URL,TITLE,MAIN_CATEGORY
2144,https://2b.com.eg/en/2b-ecstasy-gaming-mid-tow...,2B Ecstasy Gaming Mid Tower Case + Ecstasy Gam...,Accessories


In [18]:
data.drop_duplicates('URL', keep='first', inplace=True)

In [19]:
data.query('URL == "https://2b.com.eg/en/hp-omen-encoder-gaming-keyboard-6yw76aa-red-switch.html" ')

,URL,TITLE,MAIN_CATEGORY
1305,https://2b.com.eg/en/hp-omen-encoder-gaming-ke...,HP Omen Encoder Gaming Keyboard - 6YW76AA - Re...,Gaming


In [20]:
data.query('TITLE == "L\'avvento Casual Daypack Backpack, Lightweight Available in festival design + Free 5 Face Masks (FM001)" ')['URL']

3212    https://2b.com.eg/en/l-avvento-bg776-school-ba...
3213    https://2b.com.eg/en/l-avvento-bg776-1-school-...
3214    https://2b.com.eg/en/l-avvento-bg776-2-school-...
3215    https://2b.com.eg/en/l-avvento-bg776-3-school-...
3216    https://2b.com.eg/en/l-avvento-bg74f-school-ba...
3217    https://2b.com.eg/en/l-avvento-bg774-1-school-...
3218    https://2b.com.eg/en/l-avvento-bg774-2-school-...
3219    https://2b.com.eg/en/l-avvento-bg77s-school-ba...
3220    https://2b.com.eg/en/l-avvento-bg77n-school-ba...
3227    https://2b.com.eg/en/l-avvento-bg773-1-school-...
3228    https://2b.com.eg/en/l-avvento-bg77c-school-ba...
3229    https://2b.com.eg/en/l-avvento-bg773-2-school-...
3231    https://2b.com.eg/en/l-avvento-bg773-school-ba...
3292    https://2b.com.eg/en/l-avvento-bg77p-school-ba...
3330    https://2b.com.eg/en/l-avvento-bg76p-school-ba...
3428    https://2b.com.eg/en/l-avvento-bg77l-school-ba...
Name: URL, dtype: object

In [21]:
data.drop_duplicates('TITLE', keep='first', inplace=True)
#we say keep first because the Accessories label are put last

In [22]:
data.shape

(3603, 3)

In [23]:
data['MAIN_CATEGORY'].value_counts()

Accessories        2139
Home Appliances     418
Computers           388
Mobile & Tablet     355
Gaming              303
Name: MAIN_CATEGORY, dtype: int64

#### Still the data is highly imbalanced 

In [24]:
data['features'] = data['URL'] + data['TITLE']

In [25]:
data['features'][0]

'https://2b.com.eg/en/kioxia-transmemory-u202w-16gb-lu202w016gg4.htmlKIOXIA TransMemory U202W 16GB  LU202W016GG4'

In [26]:
features = data['features']

In [27]:
corpus = []
for i in features:
    i = re.sub('https://2b.com.eg/en/', '', i)
    i = re.sub('.html', ' ', i)
    i = re.sub('[^0-9a-zA-Z]', ' ', i)
    i = i.lower()
    corpus.append(i)

In [28]:
corpus

['kioxia transmemory u202w 16gb lu202w016gg4 kioxia transmemory u202w 16gb  lu202w016gg4',
 'a data uv250 usb flash drive 32 gb silver a data uv250 usb flash drive 32 gb   silver',
 'adata c008 capless sliding usb flash drive a data c008 capless sliding usb flash drive   64gb',
 'hikvision usb flash drive m200 std 16gb hikvision usb flash drive m200 std   16gb',
 'tp link tl wa855re 300mbps wi fi range extender with external antenna auto extender tp link wi fi range extender 300mbps  tl wa855re ',
 'canon pixma mg2540 all in one printer canon pixma mg2540s all in one printer',
 'l avvento mp300 aromatherapy low temperature ultraviolet sterilizer with qi wireless charger l avvento  mp300  aromatherapy low temperature ultraviolet sterilizer with qi wireless charger',
 'samsung galaxy z fold2 12gb ram 256gb mystic black 5g samsung galaxy z fold2   12gb ram   256gb   mystic black   5g',
 'canon multi function ink tank printer pixma g2411 canon pixma g2411  multi function ink tank printer',

In [29]:
X= pd.Series(corpus)
y = data['MAIN_CATEGORY']

In [30]:
cv = CountVectorizer()
oversample = SMOTE()
X =cv.fit_transform(X)
X, y = oversample.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

In [33]:
models = []
models.append(('LR', LogisticRegression()))
models.append(('SGD', SGDClassifier()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('RFC', RandomForestClassifier(n_estimators = 501,
                            criterion = 'entropy')))
models.append(('MNB', MultinomialNB()))
models.append(('SVM', SVC()))

In [34]:
results = []
names = []
scoring = 'f1_micro'
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=7,shuffle=True)
    cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

LR: 0.993517 (0.002991)
SGD: 0.994141 (0.002952)
KNN: 0.922080 (0.009314)
CART: 0.977933 (0.006134)
RFC: 0.992395 (0.003633)
MNB: 0.985164 (0.003800)
SVM: 0.992894 (0.002952)


#### So , it seems that the SGD is the best model

In [35]:
sgd = SGDClassifier()

In [36]:
sgd.fit(X_train,y_train)

SGDClassifier()

In [38]:
sgd.predict(cv.transform(['http://localhost:80/api/get_category?title=Lenovo-IdeaPad+5+14ITL05+%28Intel%C2%AE+Core%E2%84%A2+i5+-+1135G7+-+8GB+-++512GBSSD+-+Intel+Iris+Xe+Graphics+-+14.0%22FHD+%29+Gray']))

array(['Computers'], dtype='<U15')

In [43]:
filename = 'finalized_model.sav'
joblib.dump(sgd, filename)

['finalized_model.sav']

In [40]:
loaded_model = joblib.load(filename)

In [41]:
loaded_model.predict(cv.transform(['http://localhost:80/api/get_category?title=Lenovo-IdeaPad+5+14ITL05+%28Intel%C2%AE+Core%E2%84%A2+i5+-+1135G7+-+8GB+-++512GBSSD+-+Intel+Iris+Xe+Graphics+-+14.0%22FHD+%29+Gray']))

array(['Computers'], dtype='<U15')

In [42]:
loaded_model.predict(cv.transform(['title=Lenovo-IdeaPad+5+14ITL05+%28Intel%C2%AE+Core%E2%84%A2+i5+-+1135G7+-+8GB+-++512GBSSD+-+Intel+Iris+Xe+Graphics+-+14.0%22FHD+%29+Gray']))

array(['Computers'], dtype='<U15')

In [44]:
cv.vocabulary_

{'kioxia': 3229,
 'transmemory': 4772,
 'u202w': 4840,
 '16gb': 241,
 'lu202w016gg4': 3386,
 'data': 2103,
 'uv250': 4891,
 'usb': 4883,
 'flash': 2577,
 'drive': 2305,
 '32': 518,
 'gb': 2715,
 'silver': 4411,
 'adata': 1225,
 'c008': 1667,
 'capless': 1758,
 'sliding': 4438,
 '64gb': 934,
 'hikvision': 2953,
 'm200': 3415,
 'std': 4558,
 'tp': 4755,
 'link': 3335,
 'tl': 4736,
 'wa855re': 4983,
 '300mbps': 492,
 'wi': 5031,
 'fi': 2548,
 'range': 4172,
 'extender': 2496,
 'with': 5046,
 'external': 2499,
 'antenna': 1301,
 'auto': 1364,
 'canon': 1752,
 'pixma': 3995,
 'mg2540': 3547,
 'all': 1265,
 'in': 3070,
 'one': 3867,
 'printer': 4067,
 'mg2540s': 3548,
 'avvento': 1376,
 'mp300': 3710,
 'aromatherapy': 1333,
 'low': 3376,
 'temperature': 4693,
 'ultraviolet': 4859,
 'sterilizer': 4564,
 'qi': 4137,
 'wireless': 5044,
 'charger': 1810,
 'samsung': 4303,
 'galaxy': 2704,
 'fold2': 2621,
 '12gb': 167,
 'ram': 4170,
 '256gb': 404,
 'mystic': 3779,
 'black': 1548,
 '5g': 880,
 'mu

In [45]:
joblib.dump(cv, "cv.pkl")

['cv.pkl']